In [24]:
import numpy as np

# Steering Vector from Cartesian Position

In [68]:

# write a function of near field steering vector
# generate a function notation template

def nf_steering(tx_pos, rx_pos, wav_len):
    """
    Calculate the near field steering vector for a single transmitter-receiver pair from Cartesian positions.

    Parameters:
    tx_pos (numpy.ndarray): The position of the transmitter.
    rx_pos (numpy.ndarray): The position of the receiver.
    wav_len (float): The wavelength of the signal.

    Returns:
    complex: The near field steering vector.

    """
    range = np.linalg.norm(tx_pos - rx_pos)
    b = np.exp(-1j*2*np.pi/wav_len*range)
    return b

# write a function of near field ULA steering vector
def nf_ULA_steering(ant_pos, obj_pos, wav_len):
    """
    Calculate the near field ULA (Uniform Linear Array) steering vector for multiple antenna-object pairs.

    Parameters:
    ant_pos (numpy.ndarray): The positions of the antennas.
    obj_pos (numpy.ndarray): The positions of the objects.
    wav_len (float): The wavelength of the signal.

    Returns:
    numpy.ndarray: The near field ULA steering vector.

    """
    rel_pos_vec = ant_pos - obj_pos
    range_vec = np.linalg.norm(rel_pos_vec, axis=1)
    b_vec = np.exp(-1j*2*np.pi/wav_len*range_vec) / range_vec
    return b_vec


# Steering Vector from Polar Position 

In [50]:
# nf steering vector polar
def nf_steering_polar(theta, range, wav_len, n_ant, spacing_ratio = 0.5):
    d = spacing_ratio * wav_len
    delta_vec = (2 * np.arange(n_ant) - n_ant) / 2
    r_vec = np.sqrt(np.power(range, 2) + \
            np.power(delta_vec, 2) * d**2 - \
            2 * range * delta_vec * d * np.cos(theta))
    b_vec = np.exp(-1j * 2 * np.pi/wav_len * r_vec)
    return b_vec

# nf steering vector using Fresnel approximation
def nf_steering_fresnel(theta, range, wav_len, n_ant, spacing_ratio = 0.5):
    d = spacing_ratio * wav_len
    delta_vec = (2 * np.arange(n_ant) - n_ant) / 2
    r_vec = range - delta_vec * d * np.cos(theta) + (0.5 * 1/range) * np.power(delta_vec * d * np.sin(theta), 2)
    b_vec = np.exp(-1j * 2 * np.pi/wav_len * r_vec)
    return b_vec

def range_fresnel_approx(theta, range, wav_len, n_ant, spacing_ratio = 0.5):
    d = spacing_ratio * wav_len
    delta_vec = (2 * np.arange(n_ant) - n_ant) / 2
    fresnel_r_vec = np.sqrt(np.power(range, 2) + \
            np.power(delta_vec, 2) * d**2 - \
            2 * range * delta_vec * d * np.cos(theta))
    r_vec = range - delta_vec * d * np.cos(theta) + (0.5 * 1/range) * np.power(delta_vec * d * np.sin(theta), 2)
    
    return fresnel_r_vec, r_vec
    

In [67]:
np.arange(3)
np.sqrt([4,3,25])
np.power([4,3,25],2)
np.dot(np.array([1,1]), np.array([3,2]).T)

theta = np.pi/3
carrier_freq = 100 * 1e9
wav_len = 3 * 1e8 / carrier_freq
n_ant = 256
spacing_ratio = 0.5
D = n_ant * spacing_ratio * wav_len
rayleigh_range = 2 * D**2 /  wav_len
fresnel_range = 0.5 * np.sqrt(D ** 3 / wav_len)

range = 10

b_original = nf_steering_polar(theta, range, wav_len, n_ant)
b_fresnel = nf_steering_fresnel(theta, range, wav_len, n_ant)

diff = b_fresnel - b_original
np.dot(diff, diff.T)

r_vec_fresnel, r_vec = range_fresnel_approx(theta, range, wav_len, n_ant)
NMSE_range = np.mean(abs(r_vec_fresnel - r_vec) / r_vec)

NMSE_range_db = 10 * np.log10(NMSE_range)
